<a href="https://colab.research.google.com/github/tuomaseerola/audio/blob/master/audio_analysis_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Music and Science – Audio Analysis Tutorial

[Tuomas Eerola](https://www.durham.ac.uk/staff/tuomas-eerola/), Durham University, Music Department, 2025.

Audio examples adapted from the [FMP Notebooks](https://www.audiolabs-erlangen.de/resources/MIR/FMP/C0/C0.html) by Meinhard Müller, which are part of [Fundamentals of Music Processing](https://www.audiolabs-erlangen.de/fau/professor/mueller/bookFMP).


In [ ]:
#PROMPT: Press the play button to set up the technical system (import libraries etc.)
import os
import numpy as np
import librosa
import librosa.display
import IPython.display as ipd
from matplotlib import pyplot as plt
%matplotlib inline
print("Librosa:")
print(librosa.__version__)

# 1. Read audio files

Here are a few audio files that come with Librosa. We can use any of them in the subsequent sections. Just remove the hashtag in front of the line to get that audio file.

The following code also shows how to select only a _segment_ of an audio file. This is done by keywords `offset` and `duration`. Offset specifies where you want to start the segment (in seconds) and duration (also in seconds) is self-explanatory.

In [ ]:
#filename = librosa.ex('trumpet')   # Mihai Sorohan - Monophonic trumpet recording
#filename = librosa.ex('brahms')    # Hungarian Dance #5
filename = librosa.ex('choice')    # A short drum and bass loop
#filename = librosa.ex('fishin')    # Karissa Hobbs - Let’s Go Fishin’ A folk/pop song with verse/chorus/verse structure and vocals
#filename = librosa.ex('nutcracker')# Tchaikovsky - Dance of the Sugar Plum Fairy
#filename = librosa.ex('vibeace')   # 60-second clip
x, sr = librosa.load(filename, duration = 20) # if you want to start the segment from a specific point, add offset = [2 for example] in the command

plt.figure(figsize=(12, 4))
librosa.display.waveshow(y=x,sr=sr)
ipd.display(ipd.Audio(data=x, rate=sr))

---
### Learning Task 1
---
Try out some of the audio examples yourself. To select one, remove the hashtag (#) from the beginning of the line. NB. Only one audio example should be run at a time, so make sure that the rest of the 'filename' commands have a # in front of them.

## 1.1 Load audio files from online sources
This is how to load example sound files from Github pages (or anywhere from online, if you know the URL).

In [ ]:
import soundfile as sf
import io
from six.moves.urllib.request import urlopen

url = "https://raw.githubusercontent.com/tuomaseerola/audio/master/clar_one_note.wav"
#url = "https://raw.githubusercontent.com/tuomaseerola/audio/master/trumpet_one_note.wav"
#url = "https://raw.githubusercontent.com/tuomaseerola/audio/master/harp_one_note.wav"
#url = "https://raw.githubusercontent.com/tuomaseerola/audio/master/harpsichord_one_note.wav"

y, sr = sf.read(io.BytesIO(urlopen(url).read()))

plt.figure(figsize=(12, 4))
librosa.display.waveshow(y = y, sr = sr, color='purple')
ipd.display(ipd.Audio(data = y, rate = sr))

## 1.2 Load audio files from your hardrive / google drive (Optional)
It is also possible to load any audio you have on your hard drive to the workspace of the notebook by clicking the folder icon (on the left) and choosing the file, waiting for the file to upload, and then referring to the file. These audio files will **NOT be saved** across the sessions unless you connect your google drive with Colab, which is easy (see [External data: Local Files, Drive, Sheets and Cloud Storage](https://colab.research.google.com/notebooks/io.ipynb)).   

In [ ]:
# Use only if you have connected a google drive to the notebook
#y, sr = librosa.load('my_own_recording.wav') # if you want to start the segment from a specific point, add offset = [2 for example] in the command
#plt.figure(figsize=(12, 4))
#ipd.display(ipd.Audio(data = y, rate = sr))

# 2. Explore some acoustic features across time
First extract RMS (Root Mean Square) energy and convert this into decibels.

In [ ]:
filename = librosa.ex('brahms') #PROMPT: Plot the loudness for nutcracker and brahms excerpts. Which one has more dynamic changes?
y, sr = librosa.load(filename, duration = 20, offset=2)
ipd.display(ipd.Audio(data = y, rate = sr))

rms = librosa.feature.rms(y = y)                 # Extra dynamics (RMS)
db = librosa.amplitude_to_db(rms, ref = np.max)   # Convert into dB. Note that this is a relative measure (loudest is now 0)
times = librosa.times_like(rms)

#db = librosa.feature.spectral_centroid(y = y)      # calculate spectral centroid
#times = librosa.times_like(rms)

plt.figure(figsize=(12, 4))
plt.plot(times, db[0],color='black')
plt.xlabel('Time (seconds)')
plt.ylabel('dB')

---
### Learning Task 2
---
There is another feature called _spectral centroid_ that is often used as a proxy for perceptual brightness that you could visualise across time. Calculation of this is very simple and shown above (commented out). You could explore it or some other features from [spectral features](https://librosa.org/doc/latest/feature.html). Note that the labelling of the Y axis needs to be fixed.

# 3. Analyse pitch
This is best done with a monophonic example, although librosa has algorithm to track all pitches as well. Here we extract the fundamental frequency of the trumpet solo using a probabilistic variant of the so-called YIN method. The variant is by Mauch and Dixon (2014) and the original technique was proposed by De Cheveigne and Kawahara (2002).

In [ ]:
filename = librosa.ex('trumpet')   # Mihai Sorohan - Monophonic trumpet recording  #PROMPT: Run this code with the trumpet recording
y, sr = librosa.load(filename)
ipd.display(ipd.Audio(data=x, rate=sr))    # code added to hear audio file

# This is where the pyin algorithm is applied. fmin refers to the lower frequency threshold and fmax to higher frequence threshold.
f0, voiced_flag, voiced_probs = librosa.pyin(y=y, fmin=librosa.note_to_hz('C2'), fmax=librosa.note_to_hz('C6'))
times = librosa.times_like(f0)

# Plot the fundamental frequency
plt.figure(figsize=(12, 4))
plt.plot(times,f0,'ro:', linewidth=2)
plt.xlabel('Time')
plt.ylabel('Hz')
ax=plt.gca()
ax.yaxis.set_major_formatter(librosa.display.LogHzFormatter())
ax.set(yticks=[320,320+80,320+80*2,320+80*3,320+80*4,320+80*5])
ax.set(ylim=[300, 800])
ax.yaxis.set_major_formatter(librosa.display.LogHzFormatter())

# 4. Analyse Spectrum / Spectrogram

Spectrum is a useful representation of audio. Try out plotting the spectrogram with two different audio files from the available ones listed in Section 1.2 (trumpet sound vs example1d).



In [ ]:
filename = librosa.ex('trumpet') #PROMPT: Try this out with the trumpet audio example. What is the spectrogram telling you?

import librosa.display
import matplotlib.pyplot as plt

y, sr = librosa.load(filename)  #for the example1d file, put a # in front of this line of code

#Nfft = 512
stft = np.abs(librosa.stft(y))
freqs = librosa.fft_frequencies(sr=sr)

ipd.display(ipd.Audio(data = y, rate = sr))
plt.figure(figsize=(12, 4))
librosa.display.specshow(librosa.amplitude_to_db(stft, ref=np.max), x_axis='time', y_axis='log')

In [ ]:
## PROMPT: Look at the trumpet sound again. Take only the last note. What is the fundamental (F0)?
filename = librosa.ex('trumpet')   # Mihai Sorohan - Monophonic trumpet recording
y, sr = librosa.load(filename,offset=2.6,duration=0.75) # take the last note
stft = np.abs(librosa.stft(y)) # FFT
f0, voiced_flag, voiced_probs = librosa.pyin(y=y, fmin=librosa.note_to_hz('C2'), fmax=librosa.note_to_hz('C7')) # Estimate F0
f = np.nanmedian(f0)         # Get the Hz of the fundamental frequency for nice labels
n = librosa.hz_to_note(f)    # Convert Hz to note name
print(f)
print(n)


plt.figure(figsize=(12, 4))
# collapse across time and plot a spectrum representation (energy across frequencies)
Dmean=stft.mean(axis=1)/max(stft.mean(axis=1))
plt.plot(freqs,Dmean,color='m')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Amplitude')
plt.xlim([300, 2000])
x=np.arange(f,f*10,f) # Create x-ticks based on the F0
plt.xticks(x)


# 5. Extract onsets
This estimates the strength of possible onsets and then detects the onsets that are stronger than a threshold defined in the algorithm. There is also a beat tracking that uses the onsets to estimate beat in the music.

In [ ]:
## PROMPT: Look at the Brahms example. Take the first 20 seconds (as duration). Afterwards, have a look at the 'brahms' example as well.
#filename = librosa.ex('brahms') # vibeace, fishin, or choice
filename = librosa.ex('vibeace') # vibeace, fishin, or choice
y, sr = librosa.load(filename, duration = 20)

# 1. Calculate the strength of the onset candidates ---
o_env = librosa.onset.onset_strength(y = y, sr = sr) # calculate onset strength
times = librosa.times_like(o_env, sr = sr)

# 2. Calculate the onset locations ---
onset_frames = librosa.onset.onset_detect(onset_envelope = o_env, sr = sr)

plt.figure(figsize=(12, 4))

# PLOT
plt.plot(times, o_env, label='Onset strength')
plt.vlines(times[onset_frames], 0, o_env.max(), color='r', alpha=0.9,
           linestyle='--', label='Onsets')
plt.legend()

# 3. Beat tracking ---
# note that you can make the beat tracking more loose (0) or tight (100) with tightness or set the starting tempo in bpm
tempo, beats = librosa.beat.beat_track(y = y, sr = sr, trim = True, tightness = 90, start_bpm = 120)
# Sonify the beats
y_beats = librosa.clicks(frames = beats, sr = sr)
print(np.round(tempo,1))
combined = y[0:len(y)-sr]+y_beats[0:len(y)-sr]

librosa.display.waveshow(y=y_beats,sr=sr)
ipd.display(ipd.Audio(data=combined, rate=sr))

---
### Learning Task 3
---
Find **tempo** of the extract in Beats Per Minute (BPM).
*Tip. There is a command called* `librosa.feature.tempo` *that can be used to calculate the tempo.*

In [ ]:
## PROMPT:Calculate the approximate tempo of the brahms audio example.

tempo = librosa.feature.tempo(y = y, sr = sr)
print(np.round(tempo,1),'BPM')


Show all the candidates from tempo estimation.

In [ ]:
import matplotlib.pyplot as plt

# Compute local onset autocorrelation
hop_length = 512
oenv = librosa.onset.onset_strength(y=y, sr=sr, hop_length=hop_length)
tempogram = librosa.feature.tempogram(onset_envelope=oenv, sr=sr,
                                      hop_length=hop_length)
# Compute global onset autocorrelation
ac_global = librosa.autocorrelate(oenv, max_size=tempogram.shape[0])
ac_global = librosa.util.normalize(ac_global)
# Estimate the global tempo for display purposes
tempo = librosa.feature.tempo(onset_envelope=oenv, sr=sr,
                              hop_length=hop_length)[0]

# Plot
fig, ax = plt.subplots(figsize=(14, 4))
times = librosa.times_like(oenv, sr=sr, hop_length=hop_length)
freqs = librosa.tempo_frequencies(tempogram.shape[0], hop_length=hop_length, sr=sr)
plt.plot(freqs[1:], np.mean(tempogram[1:], axis=1),
             label='Mean local autocorrelation')
plt.axvline(tempo, color='black', linestyle='--', alpha=.8,
            label='Estimated tempo={:g}'.format(tempo))
plt.legend(frameon=True)
plt.set_xlabel=('BPM')
plt.grid(True)
ax.set_xlim(50,200)
#plt.show()